# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.


**Описание данных**
<br>Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
* *сalls* — количество звонков,
* *minutes* — суммарная длительность звонков в минутах,
* *messages* — количество sms-сообщений,
* *mb_used* — израсходованный интернет-трафик в Мб,
* *is_ultra* — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).



## Откройте и изучите файл

In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
import warnings
warnings.filterwarnings("ignore")


Откроем файл и выведем первые пять строк:

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Изучим информацию о файле:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


**Итог:**

- Данные состоят из 3214 строк и 5 столбцов.
- Предобработка не требуется по условию задания.
- Целевым признаком является столбец *is_ultra* (качественный тип данных). 
- Цель работы - построить модель, которая будет отбирать пользователей, которым стоит предложить сменить тариф на Ультра
- Перед нами стоит задача классификации
- Так как выборка не очень большая, возможно переобучение.

## Разбейте данные на выборки

**Разделим исходные данные на обучающую, валидационную и тестовую выборки.**

Разобьем выборку по принципу 3:1:1. Таким образом получаем:

- обучающеая выборка 60%
- валидационная выборка 20%
- тестовая выборка 20%

In [5]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=1)
df_valid, df_test = train_test_split(df_valid, test_size=0.5, random_state=1)

print('Размер тренериующей выборки:', df_train.shape[0])
print('Размер валидационной выборки:', df_valid.shape[0])
print('Размер тестовой выборки:', df_test.shape[0])

Размер тренериующей выборки: 1928
Размер валидационной выборки: 643
Размер тестовой выборки: 643


*Итого:*
- Обучение пройдет на данных *df_train*
- Валидация модели на данных *df_valid*
<br>Лучшая модель по валидации будет применена на данных *df_test*

## Исследуйте модели

**Исследуем качество разных моделей, меняя гиперпараметры.**

Сформируем три модели:
* дерево решений;
* случайный лес;
* логическая регрессия.

На базе обучающей, целевой и тестовой выборок формируем датасеты признаков и serias целевого признака *is_ultra*. Выведем для проверки:

In [6]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']


features_train.head()

,calls,minutes,messages,mb_used
1587,53.0,376.78,44.0,17334.12
11,45.0,344.32,13.0,19898.81
353,68.0,493.00,29.0,20021.73
350,65.0,423.06,40.0,18625.97
2708,45.0,296.26,75.0,13121.85


**Модель "Дерево решений"**

Самый важный гиперпараметр  дерева — max_depth. Перебираем разные значения параметра и сравнить качество моделей в разных вариантах. 

In [7]:
best_model_three = None
best_result_three = 0
depth_best= 0
for depth in range(1, 20):
    model_tree=DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model_tree.fit(features_train,target_train) 
    prediction_tree=model_tree.predict(features_valid)
    result_three = accuracy_score(prediction_tree,target_valid)
    print('При max_depth: ', depth, 'accuracy =', result_three)

    if result_three > best_result_three:
        depth_best= depth
        best_model_three = model_tree 
        best_result_three = result_three 
print()
print('Наибольшая точность выборки: accuracy =', best_result_three, end='')
print(', при max_depth =', depth_best )

При max_depth:  1 accuracy = 0.71850699844479
При max_depth:  2 accuracy = 0.7558320373250389
При max_depth:  3 accuracy = 0.7713841368584758
При max_depth:  4 accuracy = 0.7667185069984448
При max_depth:  5 accuracy = 0.7698289269051322
При max_depth:  6 accuracy = 0.7682737169517885
При max_depth:  7 accuracy = 0.7791601866251944
При max_depth:  8 accuracy = 0.7838258164852255
При max_depth:  9 accuracy = 0.7947122861586314
При max_depth:  10 accuracy = 0.7822706065318819
При max_depth:  11 accuracy = 0.7884914463452566
При max_depth:  12 accuracy = 0.7853810264385692
При max_depth:  13 accuracy = 0.7884914463452566
При max_depth:  14 accuracy = 0.7713841368584758
При max_depth:  15 accuracy = 0.7744945567651633
При max_depth:  16 accuracy = 0.7822706065318819
При max_depth:  17 accuracy = 0.7682737169517885
При max_depth:  18 accuracy = 0.7527216174183515
При max_depth:  19 accuracy = 0.7418351477449455

Наибольшая точность выборки: accuracy = 0.7947122861586314, при max_depth = 9


В данной модели полученная accuracy удовлетворяет поставленному условию.

**Модель "Случайный лес"**

Значения взяла больше, чем в DecisionTree, а max_depth использовала равный 9 как наименьший лучший показатель в дереве решений.

In [8]:
best_model_forest = None
best_result_forest = 0
est_best = 0


for est in range(1, 40 ):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=est,  max_depth=9) 
    model_forest.fit(features_train,target_train) 
    result_forest = model_forest.score(features_valid, target_valid)
    print('При n_estimators: ', est, 'результат =', result_forest)
    if result_forest > best_result_forest:
        est_best = est
        best_model_forest = model_forest 
        best_result_forest = result_forest 

print("При значении n_estimators = ", est_best, "; лучший результат: ", best_result_forest )

При n_estimators:  1 результат = 0.7558320373250389
При n_estimators:  2 результат = 0.7480559875583204
При n_estimators:  3 результат = 0.7776049766718507
При n_estimators:  4 результат = 0.7651632970451011
При n_estimators:  5 результат = 0.7791601866251944
При n_estimators:  6 результат = 0.7698289269051322
При n_estimators:  7 результат = 0.7853810264385692
При n_estimators:  8 результат = 0.7838258164852255
При n_estimators:  9 результат = 0.7822706065318819
При n_estimators:  10 результат = 0.7807153965785381
При n_estimators:  11 результат = 0.7916018662519441
При n_estimators:  12 результат = 0.7900466562986003
При n_estimators:  13 результат = 0.7853810264385692
При n_estimators:  14 результат = 0.7853810264385692
При n_estimators:  15 результат = 0.7869362363919129
При n_estimators:  16 результат = 0.7822706065318819
При n_estimators:  17 результат = 0.7822706065318819
При n_estimators:  18 результат = 0.7853810264385692
При n_estimators:  19 результат = 0.7807153965785381
Пр

**Логистическая регрессия.**

Так как для данной модели есть несколько разных вариантов параметров, из которых по умолчанию может использоваться один. Создадим функцию для испытания всех параметров, вдруг, при установленном нестандартном будет выходить лучшее решение.

In [9]:
#скорее всего, что-то не то делаю, но пусть будет
def df_accuracy_score(model):
    predictions_valid = model.predict(features_valid)
    return accuracy_score(target_valid, predictions_valid)

solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
def logistic_regression(solver=solvers):
    model = LogisticRegression(solver=solver, random_state=1)
    model.fit(features_train, target_train)
    return df_accuracy_score(model)

for solver in solvers:
    print(solver, ': ', logistic_regression(solver=solver))

newton-cg :  0.7200622083981337
lbfgs :  0.7200622083981337
liblinear :  0.6889580093312597
sag :  0.6640746500777605
saga :  0.6640746500777605


Лучший результат дают параметры newton-cg и lbfgs, самые распространеные и идущие по умолчанию.

In [10]:
model_log = LogisticRegression(random_state=12345, solver='lbfgs')
model_log.fit(features_train,target_train) 
result_log = model_log.score(features_valid,target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result_log)

Accuracy модели логистической регрессии на валидационной выборке: 0.7200622083981337


Сравним полученные пезультаты: 

In [11]:
print('Модель "Дерево решений":')
print('Наибольшая точность выборки: accuracy =', best_result_three, ', при max_depth =', depth_best, end='\n\n')

print('Модель "Случайный лес":')
print("При значении n_estimators = ", est_best, "; лучший результат: ", best_result_forest, end='\n\n' )
print('Логистическая регрессия:')
print("Accuracy модели логистической регрессии на валидационной выборке:", result_log, end='\n\n')

Модель "Дерево решений":
Наибольшая точность выборки: accuracy = 0.7947122861586314 , при max_depth = 9

Модель "Случайный лес":
При значении n_estimators =  11 ; лучший результат:  0.7916018662519441

Логистическая регрессия:
Accuracy модели логистической регрессии на валидационной выборке: 0.7200622083981337



**Вывод:** исходя из полученных данных, лучший результат вышел у модели "Дерево решений". 

## Проверьте модель на тестовой выборке

**"Дерево решений"**

In [12]:
model_tree=DecisionTreeClassifier(random_state=12345, max_depth=9)  #, min_samples_leaf=1, min_samples_split= 3
model_tree.fit(features_train, target_train)
accuracy_tree=model_tree.score(features_test, target_test)
print(accuracy_tree)

0.7853810264385692


**"Случайный лес"**

In [13]:
model_forest=RandomForestClassifier(random_state=12345, max_depth=9)
model_forest.fit(features_train, target_train)
accuracy_forest=model_forest.score(features_test, target_test)
print(accuracy_forest)

0.8087091757387247


**Логистическая регрессия**

In [14]:
model_log=LogisticRegression(random_state=12345)
model_log.fit(features_train, target_train)
accuracy_log=model_log.score(features_test, target_test)
print(accuracy_log)

0.7558320373250389


**Вывод:** при проверке модели на тестовой выборке лучший результат у "случайного леса". Исходя из полученного значения, можно сказать, что с вероятностью 80% мы получим правильный результат. 
<br>Эта модель склонна к переобучению, ее лучше использовать на небольших выборках. В данном случае вероятность переобучения низкая, т к разница между результатами с тестовой и валидационной выборками небольшой.

## (бонус) Проверьте модели на адекватность

*1-й способ:*

Оценим адекватность модели с помощью  DummyClassifier, для которого выбрала стратегию stratified, при которой гененрируется  случайный прогноз с учетом распределения классов обучающего набора:

In [15]:
#class sklearn.dummy.DummyClassifier(*, strategy='prior', random_state=None, constant=None)
clf = DummyClassifier(strategy='stratified', random_state=0)
clf.fit(features_train, target_train)
DummyClassifier(random_state=0, strategy='stratified')
results=clf.score(features_test, target_test)

print(results)

0.6049766718506998


In [16]:
#class sklearn.dummy.DummyClassifier(*, strategy='prior', random_state=None, constant=None)
clf = DummyClassifier(strategy='most_frequent', random_state=0)
clf.fit(features_train, target_train)
DummyClassifier(random_state=0, strategy='most_frequent')
results=clf.score(features_test, target_test)

print(results)

0.6936236391912908


*2-й способ* - подсчетом большего класса в исходном файле:

In [17]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

**Итого:**
* результат DummyClassifier-а - 0.69362
* Доля большего класса тестовой выборки равна ~0.69 (Смарт)
<br>С учетом того, что полученная нами модель имеет accuracy ~0.80, т е выше, чем в двух вариантах выше, мы можем считать получившуюся у нас модель адекватной для использования.

## Общий вывод

В ходе проделанной работы было выполнено:
- Изучен файл с данными.
- Разбиты данные на три выборки: обучающая, валидационная и тестовая.
- Исследованы три модели классификации: Решающее дерево, Случайный лес и Логистическая регрессия.
- Найдены оптимальные параметры для каждой модели и выбрана одна из них для обучения модели.
- Оценена точность обученной модели.
- Оценена адекватность модели.
